## Documentation

To read more about the update API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-update.html).



## Connect to ElasticSearch

In [ ]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

## Index documents

In [ ]:
es.indices.delete(index='my_index', ignore_unavailable=True)
es.indices.create(index='my_index')

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the `document_ids` list.

In [ ]:
import json
from tqdm import tqdm


document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))

for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index='my_index', body=document)
    document_ids.append(response['_id'])

In [ ]:
document_ids

## Update API

### 1. If documents exists in the index

#### 1.1 Update an existing field

In [ ]:
from pprint import pprint

response = es.update(
    index="my_index",
    id=document_ids[0],
    script={
        "source": "ctx._source.title = params.title",
        "params": {
            "title": "New Title"
        }
    },
)
pprint(response.body)

In [ ]:
response = es.get(index='my_index', id=document_ids[0])
pprint(response.body)

#### 1.2 Add a new field

To add a new field, you can either use the `script` argument or the `doc` argument.

##### 1.2.1 Method 1 (Script)

In [ ]:
response = es.update(
    index="my_index",
    id=document_ids[0],
    script={
        "source": "ctx._source.new_field = 'dummy_value'",
    },
)
pprint(response.body)

In [ ]:
response = es.get(index='my_index', id=document_ids[0])
pprint(response.body)

##### 1.2.2 Method 2 (doc)

In [ ]:
response = es.update(
    index="my_index",
    id=document_ids[0],
    doc={
        "new_value_2": "dummy_value_2",
    },
)
pprint(response.body)

In [ ]:
response = es.get(index='my_index', id=document_ids[0])
pprint(response.body)

#### 1.3 Remove a field

In [ ]:
response = es.update(
    index="my_index",
    id=document_ids[0],
    script={
        "source": "ctx._source.remove('new_field')",
    },
)
pprint(response.body)

In [ ]:
response = es.get(index='my_index', id=document_ids[0])
pprint(response.body)

### 2. If documents doesn't exist in the index

We use `doc_as_upsert` to tell Elasticsearch that if the document does not exist, it should be inserted as a new document.

In [ ]:
response = es.update(
    index="my_index",
    id="1",
    doc={
        "book_id": 1234,
        "book_name": "A book",
    },
    doc_as_upsert=True,
)

In [ ]:
pprint(response.body)

Sure enough, we have 4 documents now instead of 3.

In [ ]:
response = es.count(index='my_index')
response['count']